<a href="https://colab.research.google.com/github/david-vanderhaar/story_lorry/blob/main/story_lorry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!cat /etc/os-release
# !pwd
# !calibre
# import os
# os.system('ebook-convert --help')

PRETTY_NAME="Ubuntu 22.04.3 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.3 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [ ]:
# install ebook-convert
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt install calibre

!pip install PyPDF2
!pip install weasyprint
!pip install selenium

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following package was automatically installed and is no longer required:
  libfuse2
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  avahi-daemon bind9-host bind9-libs calibre-bin dictionaries-common dmidecode
  dosfstools eject fdisk fonts-dejavu-core fonts-droid-fallback
  fonts-liberation2 fonts-lyx fonts-noto-mono fonts-urw-base35 fuse3 gdisk
  geoclue-2.0 ghostscript glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas gsfonts hunspell-en-us
  iio-sensor-proxy imagemagick imagemagick-6-common imagemagick-6.q16
  libatasmart4 libavahi-core7 libavahi-glib1 libblockdev-crypto2
  libblockdev-fs2 libblockdev-loop2 libblockdev-part-err2 libblockdev-part2
  libblockdev-swap2 libblockdev-utils2 libblockdev2 libchm1 libdaemon0
  libdjvulibre-text libdjvulibre21 libevdev2 libfdisk1 libfftw3-double3
  libfuse

In [ ]:
# Imports
import html
import requests
import re
import os
from bs4 import BeautifulSoup
from google.colab import files
from PyPDF2 import PdfFileMerger, PdfFileReader
from IPython.core.display import display, HTML
import weasyprint
from selenium import webdriver
import time

In [ ]:
# PREVIEW = True
PREVIEW = False
FILE_OUTPUT_TYPES = [
  '.epub',
]

LORD_DUNSANY_CONFIG = {
    'document_name': 'Collected Fictions of Lord Dunsany.pdf',
    # 'homepage_url': 'https://www.poetryfoundation.org/collections/155298/poetry-and-racial-justice-and-equality',
    # 'homepage_url': 'https://www.poetryfoundation.org/collections/152025/an-introduction-to-modernism',
    # 'soup_find_all_anchors_kwargs': {'href': re.compile("/poems/\d+")},
    'target_url_range': (None, 2),
    'soup_find_main_content_args': ('body'),
    'soup_find_title_content_args': ('h2'),
    'clean_urls': False,
    'manual_target_urls': [
        'https://www.gutenberg.org/cache/epub/8395/pg8395-images.html',
        'https://www.gutenberg.org/cache/epub/8183/pg8183-images.html',
    ],
    # 'add_title': True,
    'add_title': False,
    'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'Collected Fictions of Lord Dunsany',
        '--authors': 'Lord Dunsany'
        # '--cover': 'cover.png'
    }
}

POETRY_FOUNDATION_CONFIG = {
    'document_name': 'a_racial_justice_poetry_collection.pdf',
    'homepage_url': 'https://www.poetryfoundation.org/collections/155298/poetry-and-racial-justice-and-equality',
    # 'homepage_url': 'https://www.poetryfoundation.org/collections/152025/an-introduction-to-modernism',
    'soup_find_all_anchors_kwargs': {'href': re.compile("/poems/\d+")},
    'target_url_range': (None, 2),
    'soup_find_main_content_args': ('div', {'class': 'c-feature'}),
    'soup_find_title_content_args': ('h1', {'class': 'c-hdgSans'}),
    'clean_urls': False,
    # 'add_title': True,
    'add_title': False,
    'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'A Racial Justice Poetry Collection',
        '--authors': 'Poetry Foundation'
        # '--cover': 'cover.png'
    }
}

SUNDAY_STORIES_CONFIG = {
    'document_name': 'vol_1_brooklyn.pdf',
    'homepage_url': 'http://vol1brooklyn.com/sunday-stories/',
    'soup_find_all_anchors_kwargs': {'href': re.compile("/sunday-stories-")},
    'target_url_range': (None, 1),
    'soup_find_main_content_args': ('div', {'class': 'entry-content'}),
    'soup_find_title_content_args': ('title',),
    'clean_urls': False,
    'add_title': True,
    'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'Temp Title',
        '--authors': 'Temp Author',
        '--cover': 'star.png'
    }
}

INFINITY_PLUS_CONFIG = {
    'document_name': 'infinity_plus_a_through_f.pdf',
    'homepage_url': 'http://www.infinityplus.co.uk/stories/indexaf.htm',
    'soup_find_all_anchors_kwargs': {'onmouseover': re.compile('MM_displayStatusMsg')},
    'target_url_range': (None, None),
    'soup_find_main_content_args': ('table',),
    'soup_find_title_content_args': ('title',),
    'clean_urls': True,
    'clean_url': lambda url: 'http://www.infinityplus.co.uk/stories/' + url.split('/')[-1],
    'clean_main_content': {
      'strip_src_attr': True,
    },
    'target_main_content': lambda soup: soup[1].find_all('tr')[0].find_all('td')[1],
    'add_title': True,
    'metadata': {
        '--title': 'Temp Title',
        '--authors': 'Temp Author'
    }
}

THE_ABBEY_OF_MISRULE_CONFIG = {
    'document_name': 'the_abbey_of_misrule__holy_wells.pdf',
    'homepage_url': 'https://paulkingsnorth.substack.com/t/holy-wells',
    # 'soup_find_all_anchors_kwargs': {'href': re.compile("/poems/\d+")},
    'soup_find_all_anchors_kwargs': {
        'href': re.compile("/paulkingsnorth.substack.com/p/[\w-]+"),
        'data-testid': re.compile("post-preview-title")
    },
    'target_url_range': (None, None),
    'soup_find_main_content_args': ('div', {'class': 'available-content'}),
    'soup_find_title_content_args': ('div', {'class': 'pullquote'}),
    'use_selenium': True,
    'reverse_order': True,
    'clean_urls': False,
    'add_title': True,
    # 'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'Holy Wells - The Abbey Of Misrule',
        '--authors': 'Paul Kingsworth'
    }
}

THE_MACHINE_OF_MISRULE_CONFIG = {
    'document_name': 'the_abbey_of_misrule__the_machine.pdf',
    'homepage_url': 'https://paulkingsnorth.substack.com/t/the-machine',
    'soup_find_all_anchors_kwargs': {
        'href': re.compile("/paulkingsnorth.substack.com/p/[\w-]+"),
        'data-testid': re.compile("post-preview-title")
    },
    'target_url_range': (None, None),
    'soup_find_main_content_args': ('div', {'class': 'available-content'}),
    'soup_find_title_content_args': ('h1', {'class': 'post-title'}),
    'use_selenium': True,
    'reverse_order': True,
    'clean_urls': False,
    'add_title': True,
    # 'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'The Machine - The Abbey Of Misrule',
        '--authors': 'Paul Kingsworth'
    }
}

THE_AUDACITY_OF_DESPAIR_CONFIG = {
    'document_name': 'THE_AUDACITY_OF_DESPAIR.pdf',
    'homepage_url': 'https://paulkingsnorth.substack.com/t/the-machine',
    'soup_find_all_anchors_kwargs': {
        'href': re.compile("/paulkingsnorth.substack.com/p/[\w-]+"),
        'data-testid': re.compile("post-preview-title")
    },
    'target_url_range': (None, None),
    'soup_find_main_content_args': ('div', {'class': 'available-content'}),
    'soup_find_title_content_args': ('h1', {'class': 'post-title'}),
    'use_selenium': True,
    'reverse_order': True,
    'clean_urls': False,
    'add_title': True,
    # 'target_main_content': lambda soup: soup[0],
    'metadata': {
        '--title': 'The Audacity of Despair',
        '--authors': 'David Simon'
    }
}

TARGETS = THE_ABBEY_OF_MISRULE_CONFIG
PDF_KIT_OPTIONS = {
    'page-size': 'Letter',
    'margin-top': '0.75in',
    'margin-right': '0.75in',
    'margin-bottom': '0.75in',
    'margin-left': '0.75in',
    'encoding': "UTF-8",
    'disable-external-links': True,
    'include-in-outline': True,
    'minimum-font-size': 28,
    "enable-local-file-access": ""
}


def clean_url(url):
  return TARGETS.get('clean_url')(url)

def clean_urls(urls):
  return [clean_url(url) for url in urls]

def get_main_content_from_soup(soup):
  main_soup = soup.find_all(*TARGETS['soup_find_main_content_args'])
  if TARGETS.get('target_main_content'):
    main_soup = TARGETS.get('target_main_content')(main_soup)

  return main_soup

def get_title_content_from_soup(soup):
  try:
    return soup.find(*TARGETS['soup_find_title_content_args']).string or ' '
  except:
    return ' '
  # .replace('Sunday Stories: ', '').replace('Vol. 1 Brooklyn', '')[:-3]

def convert_pdf_to_selected_types(pdf_file, types):
  file_names = []
  for output_type in types:
    new_file_name = convert_pdf_to_type(pdf_file, output_type)
    add_metadata_to_file(new_file_name)
    file_names.append(new_file_name)

  return file_names

def convert_pdf_to_type(pdf_file, file_type):
  new_file = pdf_file.replace(".pdf", file_type)
  command = f'ebook-convert {pdf_file} {new_file}'
  # command = f'ebook-convert-helper -i pdf -o {file_type} --dir /content'
  os.system(command)
  return new_file

def add_metadata_to_file(file_name):
  options = ''
  metadata = TARGETS.get('metadata', {}).items()
  for key, value in metadata:
    options += f' {key} "{value}"'

  command = f'ebook-meta {file_name} {options}'
  os.system(command)

  return file_name

def main():
  compile_pdf_with_custom_layout()
  # compile_pdf_with_website_layout()

  if PREVIEW:
    display(HTML('article.html'))
  else:
    file_names = convert_pdf_to_selected_types(TARGETS['document_name'], FILE_OUTPUT_TYPES)
    file_names.append(TARGETS['document_name'])
    for name in file_names:
      try:
        files.download(name)
      except:
        print('failed to download ' + name)

def compile_pdf_with_custom_layout():
  html_file_name = 'article.html'
  responses = get_repsonses(get_target_urls())
  parsed_responses = parse_repsonses(responses)
  merged_html = glued_articles(parsed_responses)
  save_html_to_disk(merged_html, html_file_name)
  weasyprint.HTML(string=merged_html).write_pdf(TARGETS['document_name'])


def compile_pdf_with_website_layout():
  urls = get_target_urls()
  pdfs = create_pdfs_from_urls(urls)
  merge_pdfs(pdfs, TARGETS['document_name'])


def get_target_urls():
  if TARGETS.get('manual_target_urls'):
    urls = TARGETS['manual_target_urls']
  else:
    if TARGETS.get('use_selenium', False):
      urls = get_target_urls_from_soup_by_homepage_with_selenium(TARGETS['homepage_url'])
    else:
      urls = get_target_urls_from_soup_by_homepage(TARGETS['homepage_url'])

  if TARGETS.get('clean_urls'):
    urls = clean_urls(urls)

  if TARGETS.get('reverse_order', False):
    urls.reverse()

  print('Preview urls')
  print('count: ', len(urls))
  [print(url) for url in urls]

  if PREVIEW:
    return urls[:1]

  return urls

def get_target_urls_from_soup_by_homepage(homepage_url):
  response = requests.get(homepage_url, headers=get_headers())
  soup = get_soup(response)
  anchors = soup.find_all(**TARGETS['soup_find_all_anchors_kwargs'])
  urls = [anchor.get('href') for anchor in anchors]

  range = TARGETS['target_url_range']
  start = range[0]
  end = range[1]
  urls = urls[start:end]

  return urls

def get_target_urls_v1():
  return [
      'http://vol1brooklyn.com/2010/04/18/sunday-stories-peace-love-understanding-at-first-church-of-the-church-before-church/',
      'http://vol1brooklyn.com/2010/04/25/sunday-stories-lake-celeste-or-the-joy-of-sex/',
  ]

def get_headers():
  return {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36'
      }

def get_target_urls_from_soup_by_homepage_with_selenium(homepage_url):
  # driver = webdriver.Chrome()
  # driver = webdriver.Firefox()
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless') # ensure GUI is off
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(options=chrome_options)
  driver.get(homepage_url)
  # Automatically scroll the page
  scroll_pause_time = 2
  screen_height = driver.execute_script("return window.screen.height;")
  i = 1
  while True:
    driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
    i += 1
    time.sleep(scroll_pause_time)
    scroll_height = driver.execute_script("return document.body.scrollHeight;")
    if screen_height * i > scroll_height:
      break

  response = DictWithProperties()
  response['text'] = driver.page_source
  driver.quit()

  soup = get_soup(response)
  anchors = soup.find_all(**TARGETS['soup_find_all_anchors_kwargs'])
  urls = [anchor.get('href') for anchor in anchors]

  range = TARGETS['target_url_range']
  start = range[0]
  end = range[1]
  urls = urls[start:end]

  return urls


def get_repsonses(urls):
  successes = []
  for url in urls:
    response = requests.get(url, headers=get_headers())
    if response.status_code == 200:
      successes.append(response)

  return successes

def get_raw_html_documents_with_selenium(urls):
  driver = webdriver.Chrome()
  successes = []
  for url in urls:
    driver.get(url)

    # Automatically scroll the page
    scroll_pause_time = 2
    screen_height = driver.execute_script("return window.screen.height;")
    i = 1
    while True:
      driver.execute_script(f"window.scrollTo(0, {screen_height * i});")
      i += 1
      time.sleep(scroll_pause_time)
      scroll_height = driver.execute_script("return document.body.scrollHeight;")
      if screen_height * i > scroll_height:
        break

    response = DictWithProperties()
    response['text'] = driver.page_source
    successes.append(response)

  # Close the WebDriver session
  driver.quit()
  return successes


class DictWithProperties(dict):
  def __getattr__(self, attr):
    if attr in self:
      return self[attr]
    else:
      raise AttributeError(f"'{type(self).__name__}' object has no attribute '{attr}'")


def parse_repsonses(responses):
  return [parse(response) for response in responses]

def get_soup(response):
  return BeautifulSoup(response.text, 'html.parser')

def clean_main_content(content):
  # replacing src attr in img tags since they may be relative
  if TARGETS.get('clean_main_content'):
    if TARGETS.get('clean_main_content').get('strip_src_attr'):
      content = content.replace('src=', 'data-foo-src=')
  return content

def parse(response):
  soup = get_soup(response)
  main_content = get_main_content_from_soup(soup)
  title_content = get_title_content_from_soup(soup)
  title = '<h1><b>' + title_content + '</b></h1>'
  page_break = '<div style="page-break-after: always; min-height: 1px">&nbsp</div>'

  article_parts = []

  if TARGETS.get('add_title'):
    article_parts.append(str(title))
  article_parts.append(clean_main_content(str(main_content)))
  article_parts.append(str(page_break))
  article = ''.join(article_parts)
  return {
      "soup": soup,
      "html_string": article,
  }

def save_html_to_disk(html, file_name):
  Func = open(file_name,"w")
  Func.write(html)
  Func.close()

def glued_articles(parsed):
  return '<br />'.join([item['html_string'] for item in parsed])

def merge_pdfs(file_names, output_file_name):
  merger = PdfFileMerger(False)
  merger.add_metadata(FINAL_DOCUMENT_METADATA)
  for name in file_names:
    merger.append(PdfFileReader(name, 'rb'))
  merger.write(output_file_name)
  merger.close()

def create_pdfs_from_urls(urls):
  pdf_file_name = 'out.pdf'
  pdfs = []
  for index in range(0, len(urls)):
    file_name = f'{index}_{pdf_file_name}'
    url = urls[index]
    try:
      weasyprint.HTML(url).write_pdf(file_name)
      pdfs.append(file_name)
    except:
      print('unable to convert page to pdf')
  return pdfs

main()


Preview urls
count:  39
https://paulkingsnorth.substack.com/p/the-living-water
https://paulkingsnorth.substack.com/p/the-spring-and-the-cave
https://paulkingsnorth.substack.com/p/ocean-of-coins
https://paulkingsnorth.substack.com/p/hubcaps-and-lace
https://paulkingsnorth.substack.com/p/witches-in-the-woods
https://paulkingsnorth.substack.com/p/mountain-gothic
https://paulkingsnorth.substack.com/p/bridget-and-the-eel
https://paulkingsnorth.substack.com/p/back-to-basics
https://paulkingsnorth.substack.com/p/all-is-ruin
https://paulkingsnorth.substack.com/p/some-deep-peace
https://paulkingsnorth.substack.com/p/the-one-that-got-away
https://paulkingsnorth.substack.com/p/suffer-little-children
https://paulkingsnorth.substack.com/p/secrets-of-the-city
https://paulkingsnorth.substack.com/p/green-natives
https://paulkingsnorth.substack.com/p/silent-mystery
https://paulkingsnorth.substack.com/p/what-lies-beneath
https://paulkingsnorth.substack.com/p/feichin-rumours
https://paulkingsnorth.substa

failed to download the_abbey_of_misrule__holy_wells.epub


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>